In [0]:
%sql
CREATE FUNCTION dataops.libs.create_catalog(catalog_name STRING)
  RETURNS STRING
  LANGUAGE PYTHON
  AS $$
    catalog_name = catalog_name.replace("-", "_")
    if catalog_name not ends with "_prd", ""
    if catalog_name not in spark.catalog.listDatabases():
      spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog_name}")
    def create_catalog(catalog_name):
      return "Hello " + name + "!"

    return greet(s) if s else None
  $$

In [0]:
def create_catalog(catalog_name,spark=spark):
  catalog_name = catalog_name.replace("-", "_")
  spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog_name}")
  spark.sql(f)
  spark.sql(f"USE CATALOG {catalog_name}")